In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os

from data_utils import SegItemListCustom, ImageListVertical
from data import data_dirs as data_env, get_label_with_context

from result_visualization import show_results

from fastai.vision import (
    Learner, load_learner,
    open_image, open_mask,
    ImageList,
    get_transforms, imagenet_stats,
    ResizeMethod, DatasetType,
    plt,   # matplotlib.pyplot
    nn,    # torch.nn
    optim  # torch.optim
)
from fastai.callbacks.mem import PeakMemMetric

from torchvision.utils import save_image as t_save

import torch

import cv2

import numpy as np

import warnings
warnings.filterwarnings("ignore")

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

AREA_THRESHOLD = 1500

In [2]:
learner = load_learner("/home/dmitri/Documents/model_weights/linknet34/", "linknet34_osp_full_e25_exp_without_logit_1e-03_dropout_10")
# learner = learner.load("/home/dmitri/Documents/model_weights/linknet34/linknet34_ops_full_e100_wghts", )

In [3]:
for unlabeled_image in data_env.get_subdir("train images raw").iterdir():
    img = open_image(unlabeled_image).resize(256)
    label_name = get_label_with_context(unlabeled_image)
    label = learner.predict(img)
    lable_np_array = label[1].numpy() * 254
    
    blur = cv2.GaussianBlur(lable_np_array.astype(np.uint8),(5,5),0).squeeze(0)
    ret,thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Contour detection
    _, contours, hierarchy = cv2.findContours( thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contour by area
    contour_list = []
    area_threshold = AREA_THRESHOLD
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > area_threshold :
            contour_list.append(contour)
    
    # Making binary mask by contour        
    countMask = np.zeros(thresh.shape, dtype='uint8')
    cv2.drawContours(countMask, contour_list, -1, (255, 255, 255), cv2.FILLED)
    
    # Restore accuracy of board between sky and other objects
    mask_rev = np.add(cv2.bitwise_not(thresh), cv2.bitwise_not(countMask))
    _,mask_rev = cv2.threshold(mask_rev, 0, 255, cv2.THRESH_BINARY)
    mask = cv2.bitwise_not(mask_rev) 
    
    t_save(torch.from_numpy(mask).to('cpu', torch.float), label_name)